In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [28]:
K = 24
days = 365

PVsize = 1700 #Watts
pvData = pd.read_csv('../data/pvwatts_hourly.csv', skiprows = 17) #From PVwatts Calculator
DC_output = pvData['DC Array Output (W)'] #DC Array Output (W)
daily_output = DC_output.iloc[:-1].values #drop final totals row
daily_output.reshape((24, 365))

pvDaysFinal = np.linspace(1, 24, 24)



[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24.]


In [17]:
8761/365

24.002739726027396